# FractalN Mushroom Classification

This notebook runs the training process in Google Colab using the main.py script.

## Setup
First, let's clone the repository and install dependencies:

In [ ]:
# Check CUDA and setup environment
!nvidia-smi

# Setup repository and dependencies
import os
if not os.path.exists('FractalN'):
    !git clone https://github.com/YOUR_USERNAME/FractalN.git
%cd FractalN

# Install dependencies with error handling
try:
    !pip install -r requirements.txt
    import subprocess
    cuda_output = subprocess.getoutput('nvcc --version')
    if 'release 11' in cuda_output:
        !pip install cupy-cuda11x
    elif 'release 12' in cuda_output:
        !pip install cupy-cuda12x
    else:
        !pip install cupy-cuda110
except Exception as e:
    print(f"Error in setup: {e}")
    !pip install numpy pandas torch torchvision cupy-cuda11x

# Setup Python path
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().resolve()))
sys.path.insert(0, str(Path.cwd().resolve() / 'src'))

## GPU Check
Let's verify GPU availability:

In [ ]:
import cupy as cp
import numpy as np
import time

def verify_gpu():
    print("GPU Verification:")
    try:
        # Basic GPU check
        device = cp.cuda.Device(0)
        print(f"GPU Device: {device.attributes['DEVICE_NAME'].decode('utf-8')}")
        print(f"Memory: {device.mem_info[1]/1024**3:.2f} GB Total")
        
        # Quick performance test
        size = 1000000
        start = time.time()
        x = cp.random.random(size)
        y = cp.random.random(size)
        z = x + y
        cp.cuda.Device().synchronize()
        print(f"Performance test completed in {time.time() - start:.3f}s")
        return True
    except Exception as e:
        print(f"GPU verification failed: {e}")
        return False

verify_gpu()

## Training
Now we'll run the training using main.py:

In [ ]:
# Run training using properly imported modules
from src.gpu_utils import configure_gpu
from src.main import train

# Configure GPU with memory pool
configure_gpu()

# Run training
train()

## Monitor GPU Usage
Add this cell to your notebook to monitor GPU usage:

In [ ]:
# Add monitoring cell
def monitor_gpu():
    if cp.cuda.is_available():
        device = cp.cuda.Device(0)
        mem_info = device.mem_info
        used_gb = (mem_info[1] - mem_info[0]) / 1024**3
        total_gb = mem_info[1] / 1024**3
        print(f"GPU Memory: {used_gb:.1f}GB / {total_gb:.1f}GB ({used_gb/total_gb*100:.1f}%)")

# Monitor GPU usage every 5 minutes during training
from IPython.display import clear_output
import time

def training_with_monitoring():
    start_time = time.time()
    while True:
        clear_output(wait=True)
        monitor_gpu()
        time.sleep(300)  # Update every 5 minutes
